In [1]:
import pandas as pd

#saved_x_tuesday.csv

df = pd.read_csv('flatten_data/saved_x_tuesday.csv',index_col=None)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.head(5)


,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,0.929828,-0.656845,0.659908,0.856356,-0.360853,0.196556,0.902478,-0.664118,0.189856,1.005288,...,1.015059,-1.675480,-1.945418,-0.317206,0.068530,-1.420136,0.970085,-1.129397,-0.661374,-1.041526
1,-0.056936,-0.360656,-0.780846,0.625369,0.308352,-1.207259,-0.028001,0.081051,-0.310865,0.426681,...,-1.340657,-1.282031,2.199889,1.313291,-0.594478,-1.589383,-0.237698,-0.849886,-0.121543,0.623533
2,0.490560,0.762371,-0.920771,0.216243,1.708066,-0.443268,0.518200,0.525918,0.370657,-0.779526,...,0.086032,-0.402748,0.652358,0.500372,0.695925,-1.376089,0.261011,0.044002,-0.479850,0.067027
3,0.994773,-0.568013,0.229133,-0.181303,-0.289275,0.438698,1.351305,-0.413223,-0.071022,-0.090384,...,-0.000397,-0.282919,-0.971175,0.087314,0.618838,1.372782,1.576300,0.011231,0.701612,1.439610
4,0.833888,-0.557214,0.995512,1.081478,-0.670258,0.533618,0.856240,-0.467297,0.942266,0.911166,...,-0.972883,-2.482897,-0.776684,0.761242,0.243920,0.604705,-0.222797,-2.829603,-0.909716,0.553585


In [2]:
y = pd.read_csv('flatten_data/saved_y_tuesday.csv',index_col=None)
y = y.loc[:, ~y.columns.str.contains('^Unnamed')]
y.head(5)

,action_label
0,0
1,0
2,0
3,0
4,0


In [8]:
from pynq import Overlay
from pynq import allocate
import pynq.lib.dma
import numpy as np

INPUT_SIZE = 240
OUTPUT_SIZE = 9

class Model:
    def __init__(self, bitfile, paramfile):
        # Initialize Overlay & DMA
        self.overlay = Overlay(bitfile)
        self.dma = self.overlay.axi_dma_0
        
        # Load weights and bias
        f = open(paramfile, "r")
        self.params = f.read().split(',')
        for i in range(len(self.params)):
            self.params[i] = np.float32(self.params[i])
        self.numofparams = len(self.params)
        
        # Initialize DMA buffer
        self.input_buffer = allocate(shape=(self.numofparams+INPUT_SIZE,), dtype=np.float32)
        for i in range (self.numofparams):
            self.input_buffer[i] = self.params[i]
        self.res = allocate(shape=(2*OUTPUT_SIZE,), dtype=np.float32)
        
    def classify(self, in_x):
        for i in range(INPUT_SIZE):
            self.input_buffer[i+self.numofparams] = in_x[i]

        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.recvchannel.transfer(self.res)
        self.dma.sendchannel.wait()
        self.dma.recvchannel.wait()
        max_index = np.argmax(self.res)
        return max_index, self.res[max_index]
        

In [9]:
import time

mlp = Model("bitstreams/mlpv4_4.bit", "mlpv4_4_stdscaler.csv")

correct = 0
total = 0
start_time = time.time()
for i in range(df.values.shape[0]):
    input_x = df.values[i]
    pred, prob = mlp.classify(input_x)
#     print(pred,prob, y.values[i])
    if (pred == y.values[i]):
        correct += 1
    total += 1
print("-----%s ms in average elapsed for each transfer-----" %(1000*(time.time()-start_time)/total))
print("TOTAL ACCURACY: ", correct/float(total))


-----3.4017927343935126 ms in average elapsed for each transfer-----
TOTAL ACCURACY:  0.9759183051364121
